In [133]:
import cv2 as cv
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
tf.test.is_gpu_available()

# Body_parts에 몸의 부위 0~15 
'''
0: 머리, 1:목
    2:오른쪽어깨, 3: 오른쪽 팔꿈치, 4: 오른쪽 손목
        5: 왼쪽어깨, 6: 왼쪽팔꿈치, 7: 왼쪽 손목
            8: 오른쪽 엉덩이, 9: 오른쪽 무릎, 10: 오른쪽 발목
                11: 왼쪽 엉덩이, 12: 왼쪽 무릎, 13: 왼쪽 발목
                    14 : 가슴, 15 : 배경
'''

BODY_PARTS = { "Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
                "Background": 15 }

# [0,1] 
# [1,2] [2,3] [3,4]
# [1,5] [5,6] [6,7]
# [1,14]
# [14,8] [8,9] [9,10]
# [14,11] [11,12] [12, 13]

POSE_PAIRS = [ ["Head", "Neck"], 
              ["Neck", "RShoulder"], ["RShoulder", "RElbow"],["RElbow", "RWrist"], 
              ["Neck", "LShoulder"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"], 
              ["Neck", "Chest"], 
              ["Chest", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], 
              ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]


# 각 파일 경로 설정
base_dir = 'C:/Users/jihun/MobileNet-Pose/PoseDetection_ML/openpose-master/models/pose/mpi/'
protoFile_path = str(base_dir)+'pose_deploy_linevec_faster_4_stages.prototxt'
weightsFile_path= str(base_dir) + 'pose_iter_160000.caffemodel'


# 파일을 불러오면서 경로에 있는 network 모델 불러오기
net = cv.dnn.readNetFromCaffe(protoFile_path, weightsFile_path)


# input Size 설정
inputWidth = 320
inputHeight = 240
inputScale=1.0/255
# Dataset을 만들때 넣을 2차원 배열 값( 좌표가 저장 됌 )
XY_arr = np.array([]) 




In [34]:
image_path = 'C:/Users/jihun/MobileNet-Pose/PoseDetection_ML/image/'
for i in os.listdir(image_path):
    image = cv.imread(image_path + str(i))
    image = cv.resize(image, (224,244))

    
    # 이미지 x,y 얻기 -> 행(shape[1]) 열(shape[0]) / opencv에서는 xy -> yx
    #frameWidth = frame.shape[1]
    #rameHeight = frame.shape[0]
    frameWidth = image.shape[1]
    frameHeight = image.shape[0]

    # 새로운 이미지를 네트워크 입력하기 위해 설정하는 구문
    inpBlob = cv.dnn.blobFromImage(image, inputScale, (inputWidth, inputHeight), (0,0,0), swapRB=False, crop=False)
    imgb = cv.dnn.imagesFromBlob(inpBlob)

    #network에 입력 
    net.setInput(inpBlob)
    output = net.forward() # 네트워크 출력 (예측값)
    
        # 키 포인트 검출 시 이미지에 그려준다
    points=[]

    for i in range(0,15): # 몸 부위가 0~15 (15개)
        # 해당 신체부위의 신뢰도를 얻는다. (i => 0: head ~ 14: Chest)
        probMap = output[0, i, :, :]

        # global 최대값 찾기
        minVal, prob, minLoc, point = cv.minMaxLoc(probMap)

        # 원래 이미지에 맞게 점 위치 변경
        x = (frameWidth * point[0]) / output.shape[3]
        y = (frameHeight * point[1]) / output.shape[2]

        '''
        해당 몸 부위에 원과 해당 숫자를 이미지에 그려준다
        '''

        # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가
        if prob > 0.1:
            cv.circle(image, (int(x), int(y)), 3, (0,255,255), thickness=-1, lineType=cv.FILLED) # 원을 그릴 곳
            cv.putText(image, "{}".format(i), (int(x),int(y)), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, lineType=cv.LINE_AA)
            points.append((int(x), int(y)))
        else : #검출했는데 부위가 없으면 None으로 
            points.append(None)

    A_part = []
    B_part = []

    for pair in POSE_PAIRS:

        # pair => [0(머리), 1(목)] [0(목), 1(오른쪽어깨)]
        partA = pair[0]
        partA = BODY_PARTS[partA]

        partB = pair[1]
        partB = BODY_PARTS[partB]

        print(partA, partB)
        print(points[partA], points[partB])

        A_part.append(points[partA])
        B_part.append(points[partB])

        #partA와 partB 사이에 선을 그어준다.
        if points[partA] and points[partB]: # 둘다 값이 있으면 
            cv.line(image, points[partA], points[partB], (0,255,0), 2)
    
    # 검출한 좌표 배열 만든 후 합치기
    arr_A = np.array(A_part[0]).flatten()
    arr_B = np.array(B_part[1:9]).flatten()
    f = np.concatenate((arr_A,arr_B), axis=0)
    
    # 만약 검출 못할 시(None) 0,0을 입력
    for i in range(len(f)):
        if f[i] == None:
            f[i] = (0,0)

        flattened_list = []

        for item in f:
            if isinstance(item, tuple):
                flattened_list.extend(item)
            else:
                flattened_list.append(item)
    
    # numpy 배열로 축적
    XY_arr = np.concatenate(([XY_arr, np.array(flattened_list)]), axis=0)
    print(XY_arr)


In [149]:
Data_arr = XY_arr.reshape(-1,18)
Dataset = pd.DataFrame(data = Data_arr, columns=['Head_X', 'Head_Y', 'Neck_X', 'Neck_Y', 
                                               'RShoulder_X', 'RShoulder_Y', 'RElbow_X','RElbow_Y', 'RWrist_X', 'RWrist_Y',
                                               'LShoulder_X', 'LShoulder_Y','LElbow_X', 'LElbow_Y', 'LWrist_X', 'LWrist_Y', 
                                               'Chest_X','Chest_Y'])

In [150]:
Dataset

,Head_X,Head_Y,Neck_X,Neck_Y,RShoulder_X,RShoulder_Y,RElbow_X,RElbow_Y,RWrist_X,RWrist_Y,LShoulder_X,LShoulder_Y,LElbow_X,LElbow_Y,LWrist_X,LWrist_Y,Chest_X,Chest_Y
0,152.0,146.0,158.0,105.0,183.0,146.0,195.0,178.0,164.0,105.0,183.0,138.0,195.0,178.0,146.0,81.0,91.0,48.0
1,115.0,48.0,109.0,97.0,79.0,97.0,54.0,89.0,134.0,89.0,158.0,89.0,183.0,89.0,115.0,122.0,103.0,154.0
2,146.0,16.0,158.0,81.0,67.0,130.0,109.0,138.0,103.0,65.0,176.0,138.0,134.0,138.0,0.0,0.0,0.0,0.0
3,183.0,24.0,134.0,97.0,128.0,162.0,158.0,170.0,195.0,105.0,213.0,170.0,195.0,130.0,164.0,146.0,152.0,195.0


In [151]:
# 이미지를 보려면 해당 실행 
cv.imshow('Pose-Detection', image)
cv.waitKey()
cv.destroyAllWindows()